In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Apple-Stock-Price-Prediction'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
data = pd.read_csv('./data/data.csv')

In [3]:
data = torch.from_numpy(np.array(data['Open'].tolist()))

In [4]:
data = data.view(1,-1)

In [5]:
data_input = data[:1,:-1]
data_target = data[:1,1:]

In [6]:
data_input

tensor([[ 27.1475,  27.1425,  26.8475,  ..., 150.2300, 149.8000, 145.0300]],
       dtype=torch.float64)

In [7]:
data_target

tensor([[ 27.1425,  26.8475,  26.8525,  ..., 149.8000, 145.0300, 147.4400]],
       dtype=torch.float64)

In [8]:
class Model(Module):
    def __init__(self,hidden_lstm=16,hidden_linear=16,activation=ReLU):
        super().__init__()
        self.activation = activation()
        self.hidden = hidden_lstm
        self.lstm1 = LSTMCell(1,hidden_lstm)
        self.lstm2 = LSTMCell(hidden_lstm,hidden_lstm)
        self.lstm3 = LSTMCell(hidden_lstm,hidden_lstm)
        self.linear1 = Linear(hidden_lstm,hidden_linear)
        self.linear1batchnorm = BatchNorm1d(hidden_linear)
        self.linear2 = Linear(hidden_linear,hidden_linear)
        self.linear2batchnorm = BatchNorm1d(hidden_linear)
        self.linear3 = Linear(hidden_linear,hidden_linear)
        self.linear3batchnorm = BatchNorm1d(hidden_linear)
        self.linear4 = Linear(hidden_linear,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.shape[0]
        h_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        c_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        h_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        c_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        h_t3 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        c_t3 = torch.zeros(batch_size,self.hidden,dtype=torch.float32)
        h_t1,c_t1 = h_t1.to(device),c_t1.to(device)
        h_t2,c_t2 = h_t2.to(device),c_t2.to(device)
        h_t3,c_t3 = h_t3.to(device),c_t3.to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1,c_t1 = h_t1.to(device),c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2,c_t2 = h_t2.to(device),c_t2.to(device)
            h_t3,c_t3 = self.lstm3(h_t2,(h_t3,c_t3))
            h_t3,c_t3 = h_t3.to(device),c_t3.to(device)
            pred = self.activation(self.linear1batchnorm(self.linear1(h_t3)))
            pred = self.activation(self.linear2batchnorm(self.linear2(pred)))
            pred = self.activation(self.linear3batchnorm(self.linear3(pred)))
            pred = self.linear4(pred)
            preds.append(pred)
        for i in range(future):
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1,c_t1 = h_t1.to(device),c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2,c_t2 = h_t2.to(device),c_t2.to(device)
            h_t3,c_t3 = self.lstm3(h_t2,(h_t3,c_t3))
            h_t3,c_t3 = h_t3.to(device),c_t3.to(device)
            pred = self.activation(self.linear1batchnorm(self.linear1(h_t3)))
            pred = self.activation(self.linear2batchnorm(self.linear2(pred)))
            pred = self.activation(self.linear3batchnorm(self.linear3(pred)))
            pred = self.linear4(pred)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [9]:
model = Model()

In [10]:
optimizer = LBFGS(model.parameters(),lr=0.001)

In [11]:
criterion = MSELoss()

In [12]:
epochs = 100

In [14]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        loss = criterion(preds,data_target)
        wandb.log({'Loss':loss.item()})
        loss.backward()
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        future = 100
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds.cpu().detach().numpy()
        plt.figure(figsize=(12,7))
        n = data_input.shape[1]
        def draw(y):
            plt.plot(np.arange(n),data_target.cpu().view(-1),'r')
            plt.plot(np.arange(n,n+future),y[:,n:].cpu().view(-1),'b')
        draw(preds[0])
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Preds':cv2.imread('./img.png')})
    wandb.watch(model)
wandb.finish()

  0%|          | 0/100 [00:03<?, ?it/s]


RuntimeError: CUDA error: out of memory